In [ ]:
'''
IMPORTS
'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import torch
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import os
from tqdm import tqdm

import os
import time
import torch
import numpy as np
import torch.optim as optim
from torch.utils import data
import torch.nn.functional as F
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import torch
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils import data
import itertools
import re
import random
import time
from torch.autograd import Variable
import math
from scipy.ndimage import gaussian_filter
from torch.autograd import Variable
from tqdm import tqdm
import json

import os
import torch
import dill
from e2cnn import gspaces
# from e2cnn import nn
import torch.nn as nn
import numpy as np
from torch.utils import data

import warnings
warnings.filterwarnings('ignore')

In [ ]:
disease_dict = {}

for plant_disease in os.listdir('Dataset/Dataset/'):
    disease_dict[plant_disease] = len(os.listdir('Dataset/Dataset/' + plant_disease))

disease_dict

In [ ]:
def data_loader(path, batch_size):
    
    transform = transforms.Compose(
        [transforms.Resize(255), transforms.CenterCrop(224), transforms.ToTensor()]
    )
    
    dataset = datasets.ImageFolder(path, transform = transform)
    
    indices = list(range(len(dataset)))
    
    split = int(np.floor(0.85 * len(dataset)))
    
    validation = int(np.floor(0.70 * split))
    
    np.random.shuffle(indices)
    
    train_indices, validation_indices, test_indices = (
        indices[:validation//16],
        indices[validation//16:split//16],
        indices[split//16:((split//16)+100)]
    )
    
    train_sampler = SubsetRandomSampler(train_indices)
    validation_sampler = SubsetRandomSampler(validation_indices)
    test_sampler = SubsetRandomSampler(test_indices)
    
    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=train_sampler
    )
    test_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=test_sampler
    )
    validation_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=validation_sampler
    )    
    
    return dataset, train_loader, validation_loader, test_loader

In [ ]:
dataset, train_loader, validation_loader, test_loader = data_loader(path = 'Dataset/Dataset/', batch_size = 16)

In [ ]:
''' Rotational Equivariant Unet '''

class rot_conv2d(torch.nn.Module):
    def __init__(self, input_channels, output_channels, kernel_size, stride, N, activation = True, deconv = False, last_deconv = False):
        super(rot_conv2d, self).__init__()       
        r2_act = gspaces.Rot2dOnR2(N = N)
        
        feat_type_in = nn.FieldType(r2_act, input_channels*[r2_act.regular_repr])
        feat_type_hid = nn.FieldType(r2_act, output_channels*[r2_act.regular_repr])
        if not deconv:
            if activation:
                self.layer = nn.SequentialModule(
                    nn.R2Conv(feat_type_in, feat_type_hid, kernel_size = kernel_size, stride = stride, padding = (kernel_size - 1)//2),
                    nn.InnerBatchNorm(feat_type_hid),
                    nn.ReLU(feat_type_hid)
                ) 
            else:
                self.layer = nn.R2Conv(feat_type_in, feat_type_hid, kernel_size = kernel_size, stride = stride,padding = (kernel_size - 1)//2)
        else:
            if last_deconv:
                feat_type_in = nn.FieldType(r2_act, input_channels*[r2_act.regular_repr])
                feat_type_hid = nn.FieldType(r2_act, output_channels*[r2_act.irrep(0)])
                self.layer = nn.R2Conv(feat_type_in, feat_type_hid, kernel_size = kernel_size, stride = stride, padding = 0)
            else:
                self.layer = nn.SequentialModule(
                        nn.R2Conv(feat_type_in, feat_type_hid, kernel_size = kernel_size, stride = stride, padding = 0),
                        nn.InnerBatchNorm(feat_type_hid),
                        nn.ReLU(feat_type_hid)
                    ) 
    
    def forward(self, x):
        return self.layer(x)
    
class rot_deconv2d(torch.nn.Module):
    def __init__(self, input_channels, output_channels, N, last_deconv = False):
        super(rot_deconv2d, self).__init__()
        self.conv2d = rot_conv2d(input_channels = input_channels, output_channels = output_channels, kernel_size = 4, 
                             activation = True, stride = 1, N = N, deconv = True, last_deconv = last_deconv)
        r2_act = gspaces.Rot2dOnR2(N = N)
        self.feat_type = nn.FieldType(r2_act, input_channels*[r2_act.regular_repr])
        
    def pad(self, x):
        new_x = torch.zeros(x.shape[0], x.shape[1], x.shape[2]*2 + 3, x.shape[3]*2 + 3)
        new_x[:,:,:-3,:-3][:,:,::2,::2] = x
        new_x[:,:,:-3,:-3][:,:,1::2,1::2] = x
        new_x = nn.GeometricTensor(new_x, self.feat_type)
        return new_x
    
    def forward(self, x):
        out = self.pad(x).to(device)
        return self.conv2d(out)
    
class Unet_Rot(torch.nn.Module):
    def __init__(self, input_frames, output_frames, kernel_size, N):
        super(Unet_Rot, self).__init__()
        r2_act = gspaces.Rot2dOnR2(N = N)
        self.feat_type_in = nn.FieldType(r2_act, input_frames*[r2_act.irrep(0)])
        self.feat_type_in_hid = nn.FieldType(r2_act, 32*[r2_act.regular_repr])
        self.feat_type_hid_out = nn.FieldType(r2_act, (16 + input_frames)*[r2_act.irrep(0)])
        self.feat_type_out = nn.FieldType(r2_act, output_frames*[r2_act.irrep(0)])
        
        self.conv1 = nn.SequentialModule(
            nn.R2Conv(self.feat_type_in, self.feat_type_in_hid, kernel_size = kernel_size, stride = 2, padding = (kernel_size - 1)//2),
            nn.InnerBatchNorm(self.feat_type_in_hid),
            nn.ReLU(self.feat_type_in_hid)
        )

        self.conv2 = rot_conv2d(32, 64, kernel_size = kernel_size, stride = 1, N = N)
        self.conv2_1 = rot_conv2d(64, 64, kernel_size = kernel_size, stride = 1, N = N)
        self.conv3 = rot_conv2d(64, 128, kernel_size = kernel_size, stride = 2, N = N)
        self.conv3_1 = rot_conv2d(128, 128, kernel_size = kernel_size, stride = 1, N = N)
        self.conv4 = rot_conv2d(128, 256, kernel_size = kernel_size, stride = 2, N = N)
        self.conv4_1 = rot_conv2d(256, 256, kernel_size = kernel_size, stride = 1, N = N)

        self.deconv3 = rot_deconv2d(256, 64, N)
        self.deconv2 = rot_deconv2d(192, 32, N)
        self.deconv1 = rot_deconv2d(96, 16, N, last_deconv = True)

    
        self.output_layer = nn.R2Conv(self.feat_type_hid_out, self.feat_type_out, kernel_size = kernel_size, padding = (kernel_size - 1)//2)
      

    def forward(self, x):

        x = nn.GeometricTensor(x, self.feat_type_in)
        out_conv1 = self.conv1(x)
        out_conv2 = self.conv2_1(self.conv2(out_conv1))
        out_conv3 = self.conv3_1(self.conv3(out_conv2))
        out_conv4 = self.conv4_1(self.conv4(out_conv3))

        out_deconv3 = self.deconv3(out_conv4.tensor)
        concat3 = torch.cat((out_conv3.tensor, out_deconv3.tensor), 1)
        out_deconv2 = self.deconv2(concat3)
        concat2 = torch.cat((out_conv2.tensor, out_deconv2.tensor), 1)
        out_deconv1 = self.deconv1(concat2)

        concat0 = torch.cat((x.tensor, out_deconv1.tensor), 1)
        concat0 = nn.GeometricTensor(concat0, self.feat_type_hid_out)
        out = self.output_layer(concat0)
        return out.tensor

In [ ]:
''' Train epoch function '''

def train_epoch(train_loader, model, optimizer, loss_function):
    train_mse = []

    for xx, yy in train_loader:
        
        xx = xx.to(device)
        yy = yy.to(device)

        # xx = xx.unsqueeze(1)
        # yy = yy.unsqueeze(1)

        loss = 0
        ims = []

        for y in yy.transpose(0, 1):
            y = y.unsqueeze(1)
            im = model(xx)
            xx = torch.cat([xx[:, 2:], im], 1)
            loss += loss_function(im, y)
        
        train_mse.append(loss.item()/yy.shape[1]) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_mse = round(np.sqrt(np.mean(train_mse)),5)
    return train_mse

In [ ]:
''' Eval epoch function '''

def eval_epoch(valid_loader, model, loss_function):
    valid_mse = []
    preds = []
    trues = []
    with torch.no_grad():
        for xx, yy in valid_loader:
            xx = xx.to(device)
            yy = yy.to(device)

            # xx = xx.unsqueeze(1)
            # yy = yy.unsqueeze(1)

            loss = 0
            ims = []
            for y in yy.transpose(0, 1):
                y = y.unsqueeze(1)
                im = model(xx)
                xx = torch.cat([xx[:, 2:], im], 1)
                loss += loss_function(im, y)
                ims.append(im.unsqueeze(1).cpu().data.numpy())
                
            ims = np.concatenate(ims, axis = 1)
            preds.append(ims)
            trues.append(yy.cpu().data.numpy())
            valid_mse.append(loss.item()/yy.shape[1])
        preds = np.concatenate(preds, axis = 0)  
        trues = np.concatenate(trues, axis = 0)  
        valid_mse = round(np.sqrt(np.mean(valid_mse)), 5)
    return valid_mse, preds, trues

In [ ]:
''' Test epoch function '''

def test_epoch(valid_loader, model, loss_function):
    valid_mse = []
    preds = []
    trues = []
    with torch.no_grad():
        loss_curve = []
        for xx, yy in valid_loader:
            xx = xx.to(device)
            yy = yy.to(device)

            # xx = xx.unsqueeze(1)
            # yy = yy.unsqueeze(1)

            loss = 0
            ims = []
            
            for y in yy.transpose(0, 1):
                y = y.unsqueeze(1)
                im = model(xx)
                xx = torch.cat([xx[:, 2:], im], 1)
                mse = loss_function(im, y)
                loss += mse
                loss_curve.append(mse.item())
                ims.append(im.unsqueeze(1).cpu().data.numpy())
           
            ims = np.concatenate(ims, axis = 1)
            preds.append(ims)
            trues.append(yy.cpu().data.numpy())
            valid_mse.append(loss.item()/yy.shape[1])
            
        loss_curve = np.array(loss_curve).reshape(-1,yy.shape[1])
        preds = np.concatenate(preds, axis = 0)  
        trues = np.concatenate(trues, axis = 0)  
        valid_mse = np.mean(valid_mse)
        loss_curve = np.sqrt(np.mean(loss_curve, axis = 0))
    return valid_mse, preds, trues, loss_curve

In [ ]:
''' Model Hyperparameters '''

n_epochs = 1
learning_rate = 0.001
lr_decay = 0.9

min_mse = 100
train_mse = []
valid_mse = []
test_mse = []
times = []

In [ ]:
''' Model '''

rot_unet_model = torch.nn.DataParallel(Unet_Rot(input_frames = 3, output_frames = 1, kernel_size = 3, N = 4).to(device))

optimizer = torch.optim.Adam(rot_unet_model.parameters(), learning_rate,betas=(learning_rate, 0.999), weight_decay=4e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 1, gamma=lr_decay)
loss_fun = torch.nn.MSELoss()

In [ ]:
rot_unet_model.train()